In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator


In [78]:
file_path = 'ds/DECADQ_74_previous-1950-2022.csv'
data = pd.read_csv(file_path, delimiter=";")

In [79]:
data.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,NUM_DECADE,RR,QRR,NBRR,...,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBGREL,NBJORAG,NBJBROU
0,74001001,ABONDANCE,46.2845,6.734167,947,195001,1,78.4,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,74001001,ABONDANCE,46.2845,6.734167,947,195001,2,24.8,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,74001001,ABONDANCE,46.2845,6.734167,947,195001,3,18.6,9.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,74001001,ABONDANCE,46.2845,6.734167,947,195002,1,83.1,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,74001001,ABONDANCE,46.2845,6.734167,947,195002,2,56.4,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [80]:
data.describe()

,NUM_POSTE,LAT,LON,ALTI,AAAAMM,NUM_DECADE,RR,QRR,NBRR,RRAB,...,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBGREL,NBJORAG,NBJBROU
count,1.315210e+05,131521.000000,131521.000000,131521.000000,131521.000000,131521.000000,126731.000000,126731.000000,131511.000000,126731.000000,...,58070.000000,58070.000000,58070.000000,23946.000000,58129.000000,58129.000000,58129.000000,131511.000000,28550.000000,25922.000000
mean,7.415114e+07,46.042651,6.463336,848.726941,198927.798587,2.000783,38.182292,1.475953,9.695600,17.729551,...,3.835354,20.568538,3.637661,15.257872,2.881660,2.311669,1.551618,1.754751,0.769247,0.695201
std,1.034983e+05,0.167798,0.290198,456.104228,2143.370664,0.816389,35.969416,1.891714,2.071162,14.578754,...,3.826713,44.622368,3.760872,8.911750,4.162301,3.965041,3.440570,3.821451,1.129559,1.298074
min,7.400100e+07,45.713000,5.841667,309.000000,195001.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.405642e+07,45.901667,6.255333,470.000000,197008.000000,1.000000,10.500000,1.000000,10.000000,6.600000,...,1.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.413600e+07,46.013000,6.483333,785.000000,199309.000000,2.000000,29.400000,1.000000,10.000000,15.000000,...,1.000000,0.000000,1.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.423600e+07,46.168333,6.692833,1080.000000,200803.000000,3.000000,55.400000,1.000000,10.000000,25.400000,...,9.000000,18.000000,9.000000,22.000000,7.000000,3.000000,0.000000,0.000000,1.000000,1.000000
max,7.450000e+07,46.396500,6.970833,3845.000000,202212.000000,3.000000,407.300000,9.000000,11.000000,180.000000,...,9.000000,440.000000,9.000000,31.000000,11.000000,11.000000,11.000000,11.000000,8.000000,11.000000


In [81]:
data = data.drop(data[(data["AAAAMM"] >= 195001) & (data["AAAAMM"] <= 199912)].index)

In [82]:
# Fonction pour formater les dates AAAAMM en AAAA-MM-01
def format_date(aaaamm):
    return re.sub(r'(\d{4})(\d{2})', r'\1-\2-01', str(aaaamm))

# Liste des dates formatées
data["AAAAMM"] = [format_date(val) for val in data["AAAAMM"]]
data.rename(columns={"AAAAMM": "date"}, inplace=True)

In [83]:
data["date"]

1563      2000-01-01
1564      2000-01-01
1565      2000-01-01
1566      2000-02-01
1567      2000-02-01
             ...    
131516    2022-11-01
131517    2022-11-01
131518    2022-12-01
131519    2022-12-01
131520    2022-12-01
Name: date, Length: 51835, dtype: object

In [84]:
data.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,date,NUM_DECADE,RR,QRR,NBRR,...,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBGREL,NBJORAG,NBJBROU
1563,74001001,ABONDANCE,46.2845,6.734167,947,2000-01-01,1,1.6,1.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1564,74001001,ABONDANCE,46.2845,6.734167,947,2000-01-01,2,0.0,1.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1565,74001001,ABONDANCE,46.2845,6.734167,947,2000-01-01,3,40.2,1.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1566,74001001,ABONDANCE,46.2845,6.734167,947,2000-02-01,1,60.7,1.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1567,74001001,ABONDANCE,46.2845,6.734167,947,2000-02-01,2,56.0,1.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [85]:
data_climatique = data[['date', 'TX', 'TN', 'RR', 'NBJRR1', 'NBJRR5', 'NBJRR10', 'NBJRR30', 'NBJRR50', 'NBJRR100']]

In [89]:
data_climatique.dropna()

,date,TX,TN,RR,NBJRR1,NBJRR5,NBJRR10,NBJRR30,NBJRR50,NBJRR100
2385,2022-12-01,0.7,-3.0,31.9,3.0,2.0,2.0,0.0,0.0,0.0
2386,2022-12-01,-0.3,-5.2,38.3,5.0,2.0,1.0,0.0,0.0,0.0
2387,2022-12-01,8.2,2.1,113.7,6.0,6.0,5.0,1.0,0.0,0.0
6337,2000-01-01,3.3,-1.6,0.3,0.0,0.0,0.0,0.0,0.0,0.0
6338,2000-01-01,0.7,-5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
131516,2022-11-01,12.2,5.4,30.3,4.0,3.0,1.0,0.0,0.0,0.0
131517,2022-11-01,8.9,2.5,20.8,4.0,2.0,0.0,0.0,0.0,0.0
131518,2022-12-01,4.8,0.9,24.8,3.0,2.0,1.0,0.0,0.0,0.0
131519,2022-12-01,5.3,-0.9,35.3,4.0,1.0,1.0,0.0,0.0,0.0


In [93]:
X = data_climatique[['TX']]

In [94]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(X)

In [95]:
n_input = 12
n_features = data_scaled.shape[1]
generator = TimeseriesGenerator(data_scaled, data_scaled, length=n_input, batch_size=6)

In [96]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [97]:
model.fit(generator, epochs=200)

Epoch 1/200
8638/8638 [==============================] - 68s 8ms/step - loss: nan
Epoch 2/200
8638/8638 [==============================] - 58s 7ms/step - loss: nan
Epoch 3/200
8638/8638 [==============================] - 62s 7ms/step - loss: nan
Epoch 4/200
6850/8638 [======================>.......] - ETA: 12s - loss: nan

KeyboardInterrupt: 